# Training Process for Personalized AI Agents

In this notebook, we demonstrate how to train an AI model that can predict the behavior and communication traits of personalized agents based on user input. The model is built using a neural network (LSTM) that takes text data, such as personality traits and communication styles, and classifies them accordingly.

### Step-by-Step Training Process

#### 1. **Data Import and Preprocessing**
The first step in the training process is to **import** and **preprocess** the data. We use a CSV file (`data.csv`) that contains basic details about different agents, including:

- **Name**: The agent's name.
- **Age**: The agent's age (not directly used in training but can be a potential feature).
- **Gender**: The agent's gender (similarly, not directly used but can be incorporated in advanced models).
- **Behavior**: A high-level description of the agent's overall communication style, e.g., friendly, professional, humorous, etc.
- **Specific Traits**: A detailed description of the agent's behavior, which provides more context to their communication style.

Example Data:
```csv
Name, Age, Gender, Behavior, Specific Traits
Alice, 25, Female, friendly, "likes to provide thoughtful and empathetic responses"
Bob, 30, Male, professional, "prefers concise, to-the-point, and formal communication"
Charlie, 28, Non-binary, humorous, "uses jokes and puns in responses, but remains professional"


In [1]:
import pandas as pd

# Load the simplified data from CSV
df = pd.read_csv('data.csv')

# Display the data
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the 'Specific Traits' column
tokens = tokenizer(df['Specific Traits'].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=50)

# Display tokenized data
tokens['input_ids']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class TraitPredictionNN(nn.Module):
    def __init__(self, hidden_size=128):
        super(TraitPredictionNN, self).__init__()

        # Define the model layers
        self.embedding = nn.Embedding(30522, hidden_size)  # BERT vocab size
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, 3)  # 3 traits (Behavior)

    def forward(self, x):
        # Embedding and LSTM layers
        x = self.embedding(x)
        x, (hn, cn) = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Take the last LSTM output
        x = self.output_layer(x)
        return x

# Initialize model
model = TraitPredictionNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Convert labels to numerical values (Mapping behavior to a class)
df['Behavior_Label'] = pd.Categorical(df['Behavior']).codes

# Prepare the inputs (tokenized traits) and labels (Behavior)
X = tokens['input_ids']
y = torch.tensor(df['Behavior_Label'].values)

# Train the neural network
def train_model(model, X, y, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass
        output = model(X)

        # Calculate loss
        loss = criterion(output, y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Train the model
train_model(model, X, y, epochs=10)


In [ ]:
# Function to make predictions
def predict(model, input_text):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        tokens = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt", max_length=50)
        output = model(tokens['input_ids'])
        predicted_class = torch.argmax(output, dim=1)
        return predicted_class.item()

# Test the model
test_sentence = "I love giving thoughtful and empathetic responses."
predicted_behavior = predict(model, test_sentence)
behavior_mapping = {0: 'friendly', 1: 'professional', 2: 'humorous'}
print(f"Predicted behavior for '{test_sentence}': {behavior_mapping[predicted_behavior]}")
